# The ffsim backend

Qiskit Cold Atom includes a fermion simulator backend based on [ffsim](https://github.com/qiskit-community/ffsim), a software library for fast simulations of fermionic circuits. This backend is implemented as `FfsimBackend`, and it can be used instead of the `FermionSimulator` backend for significantly improved performance.

The ffsim simulator is not supported on Windows, and in order
for it to be available, Qiskit Cold Atom must be installed with the `ffsim` extra, e.g.
```
pip install "qiskit-cold-atom[ffsim]"
```

The following code cell shows how to use the ffsim backend to simulate a circuit with 2 species of fermions, consisting of `Hop`, `Interaction`, `Phase`, and `FermiHubbard` gates. Unlike the `FermionSimulator` backend, the ffsim backend does not compute the full unitary of the circuit, since doing so is exceedingly expensive at modest system sizes.

In [1]:
import numpy as np

from qiskit_cold_atom.fermions import (
    FermiHubbard,
    FfsimBackend,
    Hop,
    Interaction,
    Phase,
)

# initialize the ffsim backend
backend = FfsimBackend()

# set the number of orbitals and occupancies
norb = 8
nocc = norb // 4
nvrt = norb - nocc
occ_a = [1] * nocc + [0] * nvrt
occ_b = [1] * nocc + [0] * nvrt
occupations = [occ_a, occ_b]

# set parameters for fermionic gates
hopping = np.ones(norb - 1)
interaction = 1.0
mu = np.ones(norb)

# construct a circuit with some fermionic gates
circuit = backend.initialize_circuit(occupations)
circuit.append(Hop(2 * norb, hopping), list(range(2 * norb)))
circuit.append(Interaction(2 * norb, interaction), list(range(2 * norb)))
circuit.append(Phase(2 * norb, mu), list(range(2 * norb)))
circuit.append(FermiHubbard(2 * norb, hopping, interaction, mu), list(range(2 * norb)))
circuit.measure_all()

# run the circuit and retrieve the measurement counts
job = backend.run(circuit, shots=10, seed=1234, num_species=2)

# access the counts
print("counts :", job.result().get_counts())

# access the memory of individual outcomes
print("\nmemory :", job.result().get_memory())

# access the statevector
print("\nstatevector :", job.result().get_statevector())

counts : {'1100000011000000': 3, '0101000001100000': 1, '0100100010010000': 1, '0101000000110000': 2, '0011000000101000': 1, '0100100001010000': 1, '0100100010100000': 1}

memory : ['1100000011000000', '0101000001100000', '1100000011000000', '0100100010010000', '0101000000110000', '0011000000101000', '0100100001010000', '0101000000110000', '1100000011000000', '0100100010100000']

statevector : [-2.46161467e-07+1.68704669e-08j -6.04117384e-08+1.30646431e-06j
  1.90223382e-06+2.98656145e-07j  4.41803652e-06+7.16188440e-07j
  2.38367007e-06-8.78351780e-06j -1.24505782e-05-4.76017841e-06j
  2.93108274e-06-1.02685984e-05j -2.31798967e-05-9.13931758e-06j
 -2.20505050e-05+4.36290814e-05j  4.99739081e-05+3.08310440e-05j
 -1.55343605e-05-6.73505300e-06j -2.03748288e-05+3.75494476e-05j
  7.93883017e-05+5.18846577e-05j  9.27295977e-05-1.22227828e-04j
 -1.11042252e-04-9.46381829e-05j -8.23587155e-06+1.23225978e-05j
  3.11072204e-05+2.44892784e-05j  6.39535504e-05-7.01321555e-05j
 -1.22777946e-04-1

In [2]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright